# 自訂的輸入層及辨識層(Dense)

## 載入套件

In [1]:
# Keras
from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D

## 步驟1：載入 data (with mask & without mask) 資料

In [2]:
train_path='D:\Python\Python_3 機器學習應用開發 (主修)\Class 8\Homework\HW3 CNN 神經網路實作_V2\Lab 2\data'
test_path='D:\Python\Python_3 機器學習應用開發 (主修)\Class 8\Homework\HW3 CNN 神經網路實作_V2\Lab 2\data'


train_batches=ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_path,
        color_mode='grayscale',
        target_size=(150, 150),
        shuffle=True,
        classes=['with_mask','without_mask'])
            
test_batches=ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_path,
        target_size=(150, 150),
        shuffle=True,
        color_mode='grayscale',
        classes=['with_mask','without_mask'])

Found 1376 images belonging to 2 classes.
Found 1376 images belonging to 2 classes.


In [3]:
epochs=30
width,height=150,150
num_features=64


model=Sequential()

model.add(Conv2D(num_features,(3,3),activation='relu',input_shape=(width,height,1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(120,activation='relu'))
model.add(Dense(2,activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [4]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )
             
history=model.fit_generator(train_batches,epochs=25,
                           validation_data=test_batches,verbose=1,shuffle=True)       

from keras.models import load_model

model.save('mask.h5')   

C:\Users\User\AppData\Local\Temp\ipykernel_18804\235262591.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_batches,epochs=25,


Epoch 1/25
43/43 [==============================] - 53s 1s/step - loss: 0.6293 - accuracy: 0.6265 - val_loss: 0.4087 - val_accuracy: 0.8256
Epoch 2/25
43/43 [==============================] - 55s 1s/step - loss: 0.2721 - accuracy: 0.8866 - val_loss: 0.1590 - val_accuracy: 0.9324
Epoch 3/25
43/43 [==============================] - 58s 1s/step - loss: 0.1666 - accuracy: 0.9390 - val_loss: 0.0881 - val_accuracy: 0.9688
Epoch 4/25
43/43 [==============================] - 58s 1s/step - loss: 0.0893 - accuracy: 0.9666 - val_loss: 0.1035 - val_accuracy: 0.9644
Epoch 5/25
43/43 [==============================] - 58s 1s/step - loss: 0.0703 - accuracy: 0.9731 - val_loss: 0.0469 - val_accuracy: 0.9797
Epoch 6/25
43/43 [==============================] - 58s 1s/step - loss: 0.0379 - accuracy: 0.9855 - val_loss: 0.0221 - val_accuracy: 0.9913
Epoch 7/25
43/43 [==============================] - 59s 1s/step - loss: 0.0493 - accuracy: 0.9797 - val_loss: 0.0248 - val_accuracy: 0.9942
Epoch 8/25
43/43 [==

In [5]:
# 任選一張圖片，例如戴口罩
img_path = 'D:\Python\Python_3 機器學習應用開發 (主修)\Class 8\Homework\HW3 CNN 神經網路實作_V2\Lab 2\data\with_mask\7-with-mask.jpg'
# #載入圖檔，並縮放寬高為 (224, 224) 
# img = image.load_img(img_path, target_size=(224, 224))

# #加一維，變成 (1, 224, 224, 3)
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# Load Model
from tensorflow.keras.models import load_model
model = load_model("mask.h5")

# Evaluate model on test data
score = model.evaluate(train_batches,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.004516919143497944
Test accuracy: 0.9992732405662537


In [6]:
# 任選一張圖片，例如沒戴口罩
img_path = 'D:\Python\Python_3 機器學習應用開發 (主修)\Class 8\Homework\HW3 CNN 神經網路實作_V2\Lab 2\data\without_mask\53.jpg'

# Load Model
from tensorflow.keras.models import load_model
model = load_model("mask.h5")

# Evaluate model on test data
score = model.evaluate(train_batches,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.004516919143497944
Test accuracy: 0.9992732405662537


## 步驟2：進行特徵工程，將特徵縮放成(0, 1)之間

In [7]:
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.applications.resnet_v2 import decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np

In [8]:
from tensorflow.keras import layers

# Define a function that applies Gaussian blur to an image
def gaussian_blur(image):
  return tf.image.random_gaussian_noise(image, mean=0.0, stddev=1.0)

# Create an ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=gaussian_blur,
    vertical_flip=True)

# Load images from a directory
training_set = datagen.flow_from_directory('D:\Python\Python_3 機器學習應用開發 (主修)\Class 8\Homework\HW3 CNN 神經網路實作_V2\Lab 2\data')

# Define a generator function that yields batches of images and labels
def generator():
  for batch_x, batch_y in training_set:
    yield (batch_x, batch_y)
# Create a tf.data.Dataset from the generator function
dataset = tf.data.Dataset.from_generator(
    generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 256, 256, 3], [None]))


#
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_batches.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = test_batches.map(lambda x, y: (normalization_layer(x), y))

Found 1376 images belonging to 2 classes.


AttributeError: 'DirectoryIterator' object has no attribute 'map'

## 顯示 ResNet152V2 完整的模型結構

In [ ]:
base_model = ResNet152V2(weights='imagenet')
print(base_model.summary())

In [ ]:
len(base_model.layers)

## 步驟3：建立模型結構

In [ ]:
# 預先訓練好的模型 -- ResNet152V2
base_model = ResNet152V2(weights='imagenet', include_top=False)
print(base_model.summary())

# 加上自訂的辨識層(Dense)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)

# 指定自訂的輸入層及辨識層(Dense)
model = Model(inputs=base_model.input, outputs=predictions)

# 模型前段不需訓練了
for layer in base_model.layers:
    layer.trainable = False

# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
len(base_model.layers)

In [ ]:
model.summary()

## 步驟5：模型訓練

In [ ]:
# 設定快取(cache)、prefetch，以增進訓練效率
AUTOTUNE = tf.data.AUTOTUNE
normalized_ds = train_batches.cache().prefetch(buffer_size=AUTOTUNE) #prefetch: 先將下一批訓練資料取進來,以減少機器效能
normalized_val_ds = test_batches.cache().prefetch(buffer_size=AUTOTUNE)

# 模型訓練
history = model.fit(normalized_ds, validation_data = normalized_val_ds, epochs=5)

## 步驟6：繪製訓練時準確率/損失函數的變化

In [ ]:
history.history.keys()

In [ ]:
# 對訓練過程的準確率繪圖
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], 'r', label='訓練準確率')
plt.plot(history.history['val_accuracy'], 'g', label='驗證準確率')
plt.xlabel('Epoch')
plt.ylabel('準確率')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='訓練損失')
plt.plot(history.history['val_loss'], label = '驗證損失')
plt.xlabel('Epoch')
plt.ylabel('損失')
plt.legend()

## 步驟7：預測

In [ ]:
# 顯示辨識的類別
class_names = train_ds.class_names
print(class_names)

In [ ]:
# 任選一張圖片，例如玫瑰
img_path = './images_test/rose.png'
# 載入圖檔，並縮放寬高為 (224, 224) 
img = image.load_img(img_path, target_size=(224, 224))

# 加一維，變成 (1, 224, 224, 3)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# 預測
preds = model.predict(x)

# 顯示預測結果
y_pred = [round(i * 100, 2) for i in preds[0]]
print(f'預測機率(%)：{y_pred}')
print(f'預測類別：{class_names[np.argmax(preds)]}')

In [ ]:
# 任選一張圖片，例如雛菊
img_path = './images_test/daisy2.jpg'
# 載入圖檔，並縮放寬高為 (224, 224) 
img = image.load_img(img_path, target_size=(224, 224))

# 加一維，變成 (1, 224, 224, 3)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# 預測
preds = model.predict(x)

# 顯示預測結果
y_pred = [round(i * 100, 2) for i in preds[0]]
print(f'預測機率(%)：{y_pred}')
print(f'預測類別：{class_names[np.argmax(preds)]}')